<a href="https://colab.research.google.com/github/Hernan4444/Hernan4444-diplomado-sistemas-recomendadores-2020-1/blob/master/diplomado_practico_final_curatornet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico final diplomado Sistemas recomendadores

Bienvenidos a la última actividad del curso de Sistemas Recomendadores. En esta oportunidad trabajaremos con un modelo de Deep Learning desarrollado en la PUC, el cual fue diseñado para recomendaro obras de arte a usuarios que hayan realizado compras anteriormente.

El trabajo original lo podrás encontrar en el siguiente link: [Content-Based Artwork Recommendation: Integrating Painting Metadata with Neural and Manually-Engineered Visual Features](https://dars.uib.no/pubs/UMUAI2018.pdf).

Cabe señalar que en esta actividad utilizaremos una versión simplificada del trabajo original.

### Instrucciones

Este laboratorio se puede realizar en grupos de 2. Es de suma importancia que le coloquen el nombre de los participantes a continuación y que sólo uno de uds. suba el archivo .ipynb a la plataforma del curso.

Para completar el laboratorio tendrán que seguir las instrucciones de este _Jupyter Notebook_, contestando a todas las actividades marcadas.

**Nombre Integrante #1**:

**Nombre Integrante #2**:

Recuerden que este laboratorio tiene nota doble.

**Importante: Cualquier falta a estas instrucciones tendrá una penalización de 1.0 punto en la nota final.**

### Descargar datos

Lo primero que tendremos que hacer es descargar el código con el modelo ya implementado y el set de datos que trabajaremos en esta ocación.

In [0]:
%%capture
!gdown https://drive.google.com/a/uc.cl/uc?id=1OxmaqDCEPrzCHqo10P5FXZQ12AKik3e4
!tar -xvf datos_practico_final_diplomado.tar.gz
!mv diplomado/TF2nets.py .
!mv diplomado/utils.py .

---
**Actividad 1** ¿Cual es uno de los aporte que entrega los modelos de deep learning cuando se trabaja con imágenes? Comente en función de como antes era la metodología de trabajo cuando se utilizaban imágenes.

**Respuesta**

---

## Cargar librerías & datos

Ya descargando los datos y archivos necesarios, continuaremos cargando todas las librerías necesarias para el laboratorio actual.

In [0]:
import os
import numpy as np
import pandas as pd
import random
import time
import json
import pickle
from math import ceil
from sklearn.preprocessing import StandardScaler
from utils import load_embeddings_and_ids, concatenate_featmats, User, HybridScorer, VisualSimilarityHandler, VisualSimilarityHandler_ContentAndStyle, get_decaying_learning_rates, ground_truth_rank_indexes, auc_exact, plot_images, load_clusters, get_art_indexes_per_cluster
import tensorflow as tf
from TF2nets import TrainLogger, CuratorNet, train_loss_fn, test_acc_fn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn import metrics

Dentro del conjunto de datos descargados, se cuenta con los _embeddings_ de los items que fueron procesados anteriormente por nuestro equipo de super ayudantes. Recuerden que el _embedding_ de las imágenes consiste en procesar cada imagen por una Red Neuronal Convolucional (CNN), que en este caso tienen las arquitectura de ResNet50 y ResNext101.

Para los interesados, les dejamos un par de links que pueden revisar:
- [Review: ResNet — Winner of ILSVRC 2015 (Image Classification, Localization, Detection)](https://towardsdatascience.com/review-resnet-winner-of-ilsvrc-2015-image-classification-localization-detection-e39402bfa5d8)
- [Review: ResNeXt — 1st Runner Up in ILSVRC 2016 (Image Classification)](https://towardsdatascience.com/review-resnext-1st-runner-up-of-ilsvrc-2016-image-classification-15d7f17b42ac)

Recuerden que extraer características visuales con una CNN nos quedamos con la representación vectorial de la última capa _fully connected_.

---

**Actividad 2**: ¿Por qué el modelo requiere preprocesar las imágenes por una red CNN?

**Respuesta**:

---

In [0]:
resnet50 = load_embeddings_and_ids('diplomado/art/ResNet50/', 'flatten_1.npy', 'ids')
resnext101 = load_embeddings_and_ids('diplomado/art/resnext101_32x8d_wsl/', 'features.npy', 'ids.npy')

---
**Actividad 3**: ¿Qué ventaja tiene cargar datos pre-entrenados?

**Respuesta:**

---

###  Preprocesamiento de datos

En esta sección haremos un preprocesamiento a los datos para que el modelo tenga un mejor desempeño.

Al trabajar con modelos de deep learning, se recomienda normalizar los datos para que cada característica tenga media 0 y desviación estándard 1. Para ello, realizamos la siguiente operación:
$x_i = \frac{x_i - \bar{x}}{std(x)}$

Esta transformación (z-score) es aplicada a los _features_ finales que es el resultado de la concatenación de los _features_ de las arquitecturas ResNet50 y ResNext101.

Además, asignamos a cada vector de imágenes un _cluster_ específico que luego utilizaremos al generar datos negativos (_negative sampling_).

In [0]:
embedding_list = [resnet50, resnext101]

artwork_ids_set = set()
for embedding in embedding_list:
    if len(artwork_ids_set) == 0:        
        artwork_ids_set.update(embedding['index2id'])
    else:
        artwork_ids_set.intersection_update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)

featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

concat_featmat = StandardScaler().fit_transform(concat_featmat)

cluster_ids, artId2clustId = load_clusters('diplomado/art/Clustering/artworkId2clusterId(resnet50+resnext101).json', n_artworks, artwork_id2index)
n_clusters = len(set(cluster_ids))
clustId2artIndexes = get_art_indexes_per_cluster(cluster_ids, n_clusters)

Para realizar análisis de similaridad cargaremos el vector característico de cada imágen transformado por un modelo PCA con 200 características principales.

Es importante señalar que los vectores PCA200 sólo se utilizan para analizar los datos y no como _input_ del modelo.

In [0]:
pca200 = load_embeddings_and_ids(
    'diplomado/art/PCA200(resnet50+resnext101)/',
    'embeddings.npy',
    'ids.npy')


pca200_embeddings = pca200['featmat']
pca200_index2id = pca200['index2id']
pca200_id2index = pca200['id2index']

assert np.array_equal(artwork_ids, pca200_index2id)

---
**Actividad 4**: ¿Cuál es la utilidad de PCA?

**Respuesta:**

---

A continuación cargaremos todos los datos relacionados a los usuarios y su respectivo comportamiento de compras.

In [0]:
users = pickle.load(open('diplomado/art/users.pickle', 'rb'))
n_users = pickle.load(open('diplomado/art/n_users.pickle', 'rb'))
artist_ids = pickle.load(open('diplomado/art/artist_ids.pickle', 'rb'))
artistId2artworkIndexes = pickle.load(open('diplomado/art/artistId2artworkIndexes.pickle', 'rb'))

En la siguiente sección, se implementan funciones para generar los set de datos de entrenamiento. Tal como se revisó en clases, el modelo se entrena con una técnica llamada _Triplet Loss_, en la cual para cada par positivo (usuerio, ítem) buscaremos un ítem negativo, esto es, un ítem que el usuario nunca ha comprado. De esta forma, nuestro set de entrenamiento se compone de tripletas de la forma $(u, i^+, i^-)$, donde $u$ es el usuario, $i^+$ es el ejemplo positivo (que ha comprado) y $i^-$ es el ejemplo negativo (que no ha comprado).

In [0]:
_MOD = 402653189
_BASE = 92821


def hash_triple(profile, pi, ni):
    h = 0
    for x in profile:
        h = ((h * _BASE) % _MOD + x) % _MOD
    h = ((h * _BASE) % _MOD + pi) % _MOD
    h = ((h * _BASE) % _MOD + ni) % _MOD
    return h


def sanity_check_instance(instance, pos_in_profile=True, profile_set=None):
    profile, pi, ni, ui = instance
    try:
        assert 0 <= pi < n_artworks
        assert 0 <= ni < n_artworks
        assert pi != ni        
        assert not vissimhandler.same(pi,ni)
        if ui == -1: return
        
        assert 0 <= ui < n_users
        user = users[ui]
        assert all(i in user.artwork_idxs_set for i in profile)
        user_profile = user.artwork_idxs_set if profile_set is None else profile_set
        assert ni not in user_profile
        if pos_in_profile is not None:
            assert (pi in user_profile) == pos_in_profile
        spi = hybrid_scorer.get_score(ui, user.artwork_idxs, pi)
        sni = hybrid_scorer.get_score(ui, user.artwork_idxs, ni)
        assert spi > sni

    except AssertionError:
        print('profile = ', profile)
        print('pi = ', pi)
        print('ni = ', ni)
        print('ui = ', ui)
        raise


def append_instance(container, instance, **kwargs):
    global _hash_collisions
    profile, pi, ni, ui = instance
    
    h = hash_triple(profile, pi, ni)
    if h in used_hashes:
        _hash_collisions += 1
        return False
    
    if vissimhandler.same(pi, ni):
        return False
    
    sanity_check_instance(instance, **kwargs)
    container.append(instance)
    used_hashes.add(h)
    return True


def print_triple(t):
    profile, pi, ni, ui = t
    print ('profile = ', [artwork_ids[i] for i in profile])
    print ('pi = ', artwork_ids[pi])
    print ('ni = ', artwork_ids[ni])
    print ('ui = ', user_ids[ui] if ui != -1 else -1)


def print_num_samples(sampler_func):
    def wrapper(instances_container, n_samples):        
        while True:
            len_before = len(instances_container)
            sampler_func(instances_container, n_samples)
            actual_samples = len(instances_container) - len_before
            delta = n_samples - actual_samples
            print('  target samples: %d' % n_samples)
            print('  actual samples: %d' % actual_samples)
            print('  delta: %d' % (delta))
            if delta <= 0: break
            print('  ** delta > 0 -> sampling more instances again ...')
            n_samples = delta
    return wrapper

def sample_artwork_index__outside_profile(
        artists_list, clusters_list, profile_set):
    while True:
        if random.random() <= FINE_GRAINED_THRESHOLD:
            if random.random() <= 0.5:
                a = random.choice(artists_list)
                i = random.choice(artistId2artworkIndexes[a])
            else:
                c = random.choice(clusters_list)
                i = random.choice(clustId2artIndexes[c])
        else:
            c = random.randint(0, n_clusters-1)
            i = random.choice(clustId2artIndexes[c])
        if i not in profile_set: return i

@print_num_samples
def generate_samples__outside_profile__real_users(instances_container, n_samples):
    n_samples_per_user = ceil(n_samples / n_users)
    debug = 0
    for ui, user in enumerate(users):        
        profile = user.artwork_idxs
        profile_set = user.artwork_idxs_set
        artists_list = user.artist_ids
        clusters_list = user.content_cluster_ids
        n = n_samples_per_user
        user_margin = CONFIDENCE_MARGIN / len(profile)
        while n > 0:
            pi = sample_artwork_index__outside_profile(artists_list, clusters_list, profile_set)
            ni = sample_artwork_index__outside_profile(artists_list, clusters_list, profile_set)
            if pi == ni: continue
            pi_score = hybrid_scorer.get_score(ui, profile, pi)
            ni_score = hybrid_scorer.get_score(ui, profile, ni)
            if pi_score < ni_score:
                pi_score, ni_score = ni_score, pi_score
                pi, ni = ni, pi
            if pi_score < ni_score + user_margin: continue
            if append_instance(instances_container, (profile, pi, ni, ui),
                               profile_set=profile_set, pos_in_profile=False):                
                n -= 1
                if n == 0 or debug % 1000 == 0:
                    print('debug: user %d/%d : n=%d' % (ui, len(users), n), flush=True, end='\r')
                debug += 1


FINE_GRAINED_THRESHOLD = 0.7
ARTIST_BOOST = 0.2
CONFIDENCE_MARGIN = 0.18

vissimhandler = VisualSimilarityHandler(cluster_ids, pca200_embeddings)

hybrid_scorer = HybridScorer(vissimhandler, artist_ids, artist_boost=ARTIST_BOOST)


vissimhandler.count = 0
used_hashes = set()
_hash_collisions = 0
train_instances = []
test_instances = []

TOTAL_SAMPLES__TRAIN = 50000
TOTAL_SAMPLES__TEST =  TOTAL_SAMPLES__TRAIN * 0.2

---
**Actividad 5**: Dado el preprocesamiento que se ha implementado hasta el momento, ¿qué tipo de sistema recomendador se implementará? Justifique su respuesta.

**Respuesta**:

---

---
**Actividad 6** En base a la respuesta anterior, mencione una ventaja y desventaja del tipo de sistema recomendador que se implementará. Explique en detalle cada ventaja y desventaja.

**Respuesta**

---

## Sampleo de Triplets para Rankear

En esta sección utilizaremos las funciones descritas anteriormente para generar las tripletas necesarias para entrenar el modelo.

---
**Actividad 7**: Explique brevemente en sus palabras cómo funciona el modelo de ranking _Curator Net_ (modelo implementado en este laboratorio).

**Respuesta**:

---

In [0]:
%%time
print('=======================================\nSampleando tuplas de entrenamiento...')
generate_samples__outside_profile__real_users(train_instances, n_samples=TOTAL_SAMPLES__TRAIN)

print('=======================================\nSampleando tuplas de test...')
generate_samples__outside_profile__real_users(test_instances, n_samples=TOTAL_SAMPLES__TEST)

print(len(train_instances), len(test_instances))
print('hash_collisions = ', _hash_collisions)
print('visual_collisions = ', vissimhandler.count)

random.shuffle(train_instances)
train_instances.sort(key=lambda x: len(x[0]))
test_instances.sort(key=lambda x: len(x[0]))

---
**Actividad 8**
Explique con sus palabras cuál es el beneficio de utilizar sampleo de tripletas para entrenar la función de _ranking_. _Hint_: se revisó en clase.

**Respuesta**

---

### Entrenamiento del modelo

Ya teniendo todos los datos cargados y preprocesados, estamos en condiciones de entrenar nuestro modelo de _Deep Learning_.

In [0]:
def generate_minibatches(tuples, max_users_items_per_batch):
    ui_count = 0
    offset = 0
    
    batch_ranges = []
    for i, t in enumerate(tuples):
        ui_count += len(t[0]) + 3
        if ui_count > max_users_items_per_batch:
            batch_ranges.append((offset, i))
            ui_count = len(t[0]) + 3
            offset = i
            assert ui_count <= max_users_items_per_batch
    assert offset < len(tuples)
    batch_ranges.append((offset, len(tuples)))
            
    n_tuples = len(tuples)
    n_batches = len(batch_ranges)
    print('n_tuples = ', n_tuples)
    print('n_batches = ', n_batches)
    
    assert batch_ranges[0][0] == 0
    assert all(batch_ranges[i][1] == batch_ranges[i+1][0] for i in range(n_batches-1))
    assert batch_ranges[-1][1] == n_tuples
    assert sum(b[1] - b[0] for b in batch_ranges) == n_tuples
    
    profile_indexes_batches = [None] * n_batches
    profile_size_batches = [None] * n_batches
    positive_index_batches = [None] * n_batches
    negative_index_batches = [None] * n_batches
    
    for i, (jmin, jmax) in enumerate(batch_ranges):
        actual_batch_size = jmax - jmin
        profile_maxlen = max(len(tuples[j][0]) for j in range(jmin, jmax))
        profile_indexes_batch = np.full((actual_batch_size, profile_maxlen), 0, dtype=int)
        profile_size_batch = np.empty((actual_batch_size,))
        positive_index_batch = np.empty((actual_batch_size,), dtype=int)
        negative_index_batch = np.empty((actual_batch_size,), dtype=int)
        
        for j in range(actual_batch_size):
            # profile indexes
            for k,v in enumerate(tuples[jmin+j][0]):
                profile_indexes_batch[j][k] = v
            # profile size
            profile_size_batch[j] = len(tuples[jmin+j][0])        
            # positive index
            positive_index_batch[j] = tuples[jmin+j][1]
            # negative index
            negative_index_batch[j] = tuples[jmin+j][2]
            
        profile_indexes_batches[i] = profile_indexes_batch
        profile_size_batches[i] = profile_size_batch
        positive_index_batches[i] = positive_index_batch
        negative_index_batches[i] = negative_index_batch
        
    return dict(
        profile_indexes_batches = profile_indexes_batches,
        profile_size_batches    = profile_size_batches,
        positive_index_batches  = positive_index_batches,
        negative_index_batches  = negative_index_batches,
        n_batches               = n_batches,
    )


def sanity_check_minibatches(minibatches):
    profile_indexes_batches = minibatches['profile_indexes_batches']
    profile_size_batches = minibatches['profile_size_batches']
    positive_index_batches = minibatches['positive_index_batches']
    negative_index_batches = minibatches['negative_index_batches']
    n_batches = minibatches['n_batches']
    assert n_batches == len(profile_indexes_batches)
    assert n_batches == len(profile_size_batches)
    assert n_batches == len(positive_index_batches)
    assert n_batches == len(negative_index_batches)
    assert n_batches > 0
    
    for profile_indexes, profile_size, positive_index, negative_index in zip(
        profile_indexes_batches,
        profile_size_batches,
        positive_index_batches,
        negative_index_batches
    ):
        n = profile_size.shape[0]
        assert n == profile_indexes.shape[0]
        assert n == positive_index.shape[0]
        assert n == negative_index.shape[0]
        
        for i in range(n):
            assert positive_index[i] != negative_index[i]
            psz = int(profile_size[i])
            m = profile_indexes[i].shape[0]
            assert psz <= m
            for j in range(psz, m):
                assert profile_indexes[i][j] == 0

Como este modelo no es de clasificación o regresión, el error que vamos a calcular para optimizar es distinto. Es por eso que creamos nuestra propia forma de calcular el error, la cual está contenida en la función `train_loss_fn` y que es llamada dentro de la función `train_step`.

In [0]:
def train_step(model, opt, x, embs):
    with tf.GradientTape() as tape:
        uv, pv, nv = model(x, embs)
        loss = train_loss_fn((uv, pv, nv), model)
    grads = tape.gradient(loss, model.trainable_weights)
    opt.apply_gradients(zip(grads, model.trainable_weights))
    return loss

---
**Actividad 9:** Explique qué busca el modelo cuando procesa el embedding de usuario y de ítems. _Hint_: La respuesta está en interpretar la función de pérdida correctamente.

**Respuesta:**

---

En las celdas que siguen, se implementan las funciones necesarias para entrenar el modelo. La función `train_loop` implementa lo que sucede en cada paso del descenso de gradiente.

In [0]:
def train_loop(model, opt, train_mb, test_mb, patience=3, **kwargs):

    # ========= CHECKPOINTING ============
    trainlogger = TrainLogger(kwargs['model_path'] + 'train_logs.csv')
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, kwargs['model_path'], max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)

    if manager.latest_checkpoint:
        print("Modelo restaurado de {}".format(manager.latest_checkpoint))
    else:
        print("Modelo inicializado desde cero.")

    # ========= PREPARATIONS ============
    initial_test_acc = 0.
    for profile_indexes, positive_index, negative_index in zip(test_mb['profile_indexes_batches'],
                                                               test_mb['positive_index_batches'],
                                                               test_mb['negative_index_batches']):
        uv, pv, nv = model((profile_indexes, positive_index, negative_index), kwargs['pretrained_embeddings'])
        minibatch_test_acc = test_acc_fn((uv, pv, nv))
        initial_test_acc += minibatch_test_acc

    initial_test_acc /= kwargs['n_test_instances']
    print("Antes de entrenar: test_accuracy = {}".format(initial_test_acc))

    best_test_acc = initial_test_acc
    seconds_training = 0
    elapsed_seconds_from_last_check = 0
    checks_with_no_improvement = 0

    # ========= TRAINING ============
    print('Entrenamiento iniciado...')

    while seconds_training < kwargs['max_seconds_training']:

        for train_i, (profile_indexes,  positive_index, negative_index) in enumerate(zip(
                train_mb['profile_indexes_batches'],
                train_mb['positive_index_batches'],
                train_mb['negative_index_batches'])):

            # optimize and get training loss
            start_t = time.time()
            minibatch_train_loss = train_step(model, opt, (profile_indexes, positive_index, negative_index), kwargs['pretrained_embeddings'])
            delta_t = time.time() - start_t

            if train_i % 10 == 0:
                print('Batch training loss: {}'.format(float(minibatch_train_loss)))

            # update time tracking variables
            seconds_training += delta_t
            elapsed_seconds_from_last_check += delta_t

            # check for improvements using test set if it's time to do so
            if elapsed_seconds_from_last_check >= kwargs['min_seconds_to_check_improvement']:

                # --- testing
                test_acc = 0.
                for test_profile_indexes, test_positive_index, test_negative_index in zip(
                        test_mb['profile_indexes_batches'],
                        test_mb['positive_index_batches'],
                        test_mb['negative_index_batches'] ):
                    tuv, tpv, tnv = model((test_profile_indexes, test_positive_index, test_negative_index), kwargs['pretrained_embeddings'])
                    minibatch_test_acc = test_acc_fn((tuv, tpv, tnv))
                    test_acc += minibatch_test_acc
                test_acc /= kwargs['n_test_instances']

                print(("[Checkpoint] test_accuracy = %.7f,"
                       " check_secs = %.2f, total_secs = %.2f") % (
                          test_acc, elapsed_seconds_from_last_check, seconds_training))

                # check for improvements
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    checks_with_no_improvement = 0
                    save_path = manager.save()
                    print("   ** Mejora detectada: modelo guardado en ", save_path)
                    model_updated = True
                else:
                    checks_with_no_improvement += 1
                    model_updated = False

                # --- logging ---
                trainlogger.log_update(minibatch_train_loss,
                    test_acc, kwargs['n_train_instances'], kwargs['n_test_instances'],
                    elapsed_seconds_from_last_check, kwargs['batch_size'], opt.learning_rate.numpy(),
                    't' if model_updated else 'f')

                if checks_with_no_improvement > patience:
                    print('=== EARLY STOPPING ===')
                    return None
                # --- reset check variables
                elapsed_seconds_from_last_check = 0
    print('====== TIMEOUT. ======')

In [0]:
def train_network(train_minibatches, test_minibatches,
                  n_train_instances, n_test_instances, batch_size,
                  pretrained_embeddings,
                  user_layer_units,
                  item_layer_units,
                  model_path,
                  max_seconds_training=3600,
                  min_seconds_to_check_improvement=60,
                  patience=3):

    model = CuratorNet(
        pretrained_embedding_dim=pretrained_embeddings.shape[1],
        user_layer_units=user_layer_units,
        item_layer_units=item_layer_units
    )

    opt = tf.keras.optimizers.Adam()

    model.compile(optimizer=opt,
                  loss=None,
                  metrics=None)

    train_loop(model, opt,
               train_minibatches,
               test_minibatches,
               pretrained_embeddings=pretrained_embeddings,
               model_path=model_path,
               n_test_instances=n_test_instances,
               max_seconds_training=max_seconds_training,
               min_seconds_to_check_improvement=min_seconds_to_check_improvement,
               n_train_instances=n_train_instances,
               batch_size=batch_size,
               patience=patience)

---
**Actividad 10**: Cuando se compila el modelo, ¿por qué no se le entrega una función de perdida? y ¿qué optimizador estamos ocupando?

**Respuesta:**

---

In [0]:
train_minibatches = generate_minibatches(train_instances, max_users_items_per_batch=600*10)
sanity_check_minibatches(train_minibatches)

test_minibatches = generate_minibatches(test_instances, max_users_items_per_batch=600*10)
sanity_check_minibatches(test_minibatches)

avg_train_batch_size = ceil(np.mean([b.shape[0] for b in train_minibatches['profile_indexes_batches']]))

En la siguiente celda, entrenaremos un modelo creado inicialmente con parámetros aleatorios, al igual que los modelos revisados en clases.

Podemos revisar cómo la métrica _accuracy_ aumenta luego de entrenar el modelo con algunos datos.

In [0]:
%%time
MODEL_PATH = 'modelo_diplomado'

MINUTES = 1
train_network(train_minibatches, test_minibatches,
              len(train_instances), len(test_instances),
              batch_size=avg_train_batch_size,
              pretrained_embeddings=concat_featmat,
              user_layer_units=[300, 300, 200],
              item_layer_units=[200, 200],
              model_path=MODEL_PATH,
              max_seconds_training=60*MINUTES,
              min_seconds_to_check_improvement=60*1,
              patience=3)

## Evaluación

Teniendo nuestro modelo y entrenado, continuaremos con la evaluación de nuestro modelo.

In [0]:
def get_aucs_and_topk_recommendations(model_path, train_test_list, K=10, num_users=10):
    resnet50 = load_embeddings_and_ids('diplomado/art/ResNet50/', 'flatten_1.npy', 'ids')
    resnext101 = load_embeddings_and_ids('diplomado/art/resnext101_32x8d_wsl/', 'features.npy', 'ids.npy')
    embedding_list = [resnet50, resnext101]

    artwork_ids_set = set()
    for embedding in embedding_list:
        if len(artwork_ids_set) == 0:
            artwork_ids_set.update(embedding['index2id'])
        else:
            artwork_ids_set.intersection_update(embedding['index2id'])
    artwork_ids = list(artwork_ids_set)
    artwork_id2index = {_id: i for i, _id in enumerate(artwork_ids)}
    n_artworks = len(artwork_ids)

    featmat_list = [tmp['featmat'] for tmp in embedding_list]
    id2index_list = [tmp['id2index'] for tmp in embedding_list]
    concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

    embeddings = StandardScaler().fit_transform(concat_featmat)

    all_indexes = list(range(n_artworks))
    recommendations = []

    model = CuratorNet(pretrained_embedding_dim=embeddings.shape[1],
                       user_layer_units=[300, 300, 200],
                       item_layer_units=[200, 200])

    opt = tf.keras.optimizers.Adam(clipnorm=0.1)
    model.compile(optimizer=opt,
                  loss=None,
                  metrics=None)

    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, model_path, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint).expect_partial()

    aucs = []
    for row in tqdm(train_test_list[:num_users]):
        train_indexes = np.array([artwork_id2index[_id] for _id in row['train']])
        test_indexes_set = set(artwork_id2index[_id] for _id in row['test'])
        train_indexes_set = set(train_indexes)
        candidate_indexes = np.array([i for i in all_indexes if i not in train_indexes_set])

        user_score, candidates_score, _ = model((np.expand_dims(train_indexes, axis=0),
                                                 np.expand_dims(candidate_indexes, axis=0),
                                                 np.expand_dims(candidate_indexes, axis=0)),
                                                embeddings)

        u_score = user_score
        candidates_score = np.squeeze(candidates_score)
        result = tf.squeeze(tf.matmul(u_score, tf.transpose(candidates_score)))

        tuples = [(s, i) for i, s in zip(candidate_indexes, result)]
        tuples.sort(reverse=True)

        ranked_candidate_indexes = [t[1] for t in tuples]
        gt_indexes = ground_truth_rank_indexes(ranked_candidate_indexes, test_indexes_set)
        auc = auc_exact(gt_indexes, len(candidate_indexes))
        aucs.append(auc)
        recommendations.append([artwork_ids[idx] for i, idx in enumerate(ranked_candidate_indexes) if i < K])

        print("AUC: {:4f} ; AVG: {:4f}".format(auc, sum(aucs) / len(aucs)))

    return aucs, recommendations


def visualize_recommendation(train_test, recommendations, aucs, images_path='diplomado/images', image_cache=dict(), topk=10):
    pairs = [(auc, i) for i, auc in enumerate(aucs)]
    pairs.sort(reverse=True)
    for j in range(len(pairs)):
        i = pairs[j][1]
        print("\nauc = %f" % pairs[j][0])
        print("-------- ENTRENAMIENTO (%d) ----------" % len(train_test[i]['train']))
        plot_images(plt, image_cache, train_test[i]['train'], images_path=images_path)
        print("-------- RECOMENDACION (%d) ----------" % topk)
        plot_images(plt, image_cache, recommendations[i][:topk], images_path=images_path)
        print("-------- DATOS REALES  (%d) ----------" % len(train_test[i]['test']))
        plot_images(plt, image_cache, train_test[i]['test'], images_path=images_path)

In [0]:
test = json.load(open('diplomado/test.json'))
print('Nº usuarios de test:', len(test))

In [0]:
EVAL_MODEL_PATH = 'diplomado/modelo_preentrenado_diplomado'

La celda de a continuación puede tomar unos minutos en ejecutarse.

In [0]:
%%time
aucs, recommendations = get_aucs_and_topk_recommendations(EVAL_MODEL_PATH, test)
print("AUC Final: {:4f}".format(sum(aucs) / len(aucs)))

---
**Actividad 11:** Cuando evaluamos estamos calculando el AUC, ¿qué otra métrica sería de utilidad calcular en este problema?

**Respuesta:**

---

### Ver recomendaciones

Por último analizaremos cualitativamente qué está recomendando el modelo, analizando el historial de compras del usuario.

La sección marcada con "ENTRENAMIENTO" muestra el historial de compras de un usuario, mientras que la sección "RECOMENDACION" muestra los primero 10 ítems recomendados.

---
**Actividad 12**: Además de la visualización que se realiza en esta sección, mencione qué otro uso se le podría dar a los embeddings de usuario o ítems que genera el modelo.

**Respuesta**

---

In [0]:
visualize_recommendation(test[:10], recommendations, aucs)

---
**Actividad 13:** ¿Qué le parecen las recomendaciones con respecto a los ítems consumidos por los usuarios? **Comente en relación al contenido de las imágenes entregadas: como color, tema, que se muestra, estilo de dibujo, tipo de trazo, etc**

**Respuesta:**

---

**Actividad 14** 
El dueño de TuGalería, don Galería, cuenta con información geográfica de sus clientes y cada ubicación está catalogada como ciudad, bosque, playa, etc. Don Galería, 
le pide a Ud. que incorpore la información de la ubicación en las recomendaciones. Elabore una propuesta conceptual.

**Respuesta**:

---